# Analisi dichiarative

In [ ]:
import pandas as pd
import numpy as np
import datetime
#path = r"C:\Users\ianto\Desktop\Corso Python\file"

Creiamo una funzione per importare due dei file studiati nella lezione precedente

In [ ]:
def ImportFile():
            
    clienti = pd.read_csv(filepath_or_buffer = "Clienti.csv",
            sep = ";", 
            header = 0
            )  

    clienti["DataNascita"] =  pd.to_datetime(clienti["DataNascita"])
    
    fatture = pd.read_csv(filepath_or_buffer = "Fatture.csv",
            sep = ";",  #separtore del file
            header = None,
            names = ["NumeroFattura","Tipologia","Importo","Iva","IdCliente","Regione","DataFattura","NumeroFornitore"],
            decimal = ",",
            )
    
    fatture['DataFattura'] =  pd.to_datetime(fatture['DataFattura'],
                                             format="%d/%m/%Y"
                                            )  
    
    fatture['NumeroFornitore'] = fatture['NumeroFornitore'].astype('Int64')

    return clienti,fatture

In [ ]:
clienti,fatture = ImportFile()

In [ ]:
fatture.head(5)

In [ ]:
clienti.head(5)

In [ ]:
#Per visualizzare tutte le righe e tutte le colonne
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)

# Selezionare e filtrare i dati

Estrarre le colonne NumeroClienti, Nome e regione dal dataframe Clienti. Visualizzare solo 5 righe


In [ ]:
#SELECT TOP 5 NumeroCliente, Nome, Regione 
#FROM   Clienti

clienti[["NumeroCliente", "Nome", "Regione"]].head(5)

Indicando solo il nome del dataframe vedrò tutte le colonne

In [ ]:
#SELECT TOP 5 * 
#FROM   Clienti

clienti.head(5)

Estrarre tutte le informazioni dei clienti della regione Lazio

In [ ]:
#SELECT * 
#FROM   Clienti 
#WHERE  Regione='Lazio'

clienti.query('Regione == "Lazio"')

In [ ]:
#vecchio metodo
clienti[clienti["Regione"] == "Lazio"]

## Filtri con AND

Estrarre NumeroCliente e Nome dei clienti della regione Lazio che si chiamano Giovanni

In [ ]:
#SELECT NumeroCliente, 
#       Nome 
#FROM   Clienti 
#WHERE  Regione='Lazio'
#   AND Nome='Giovanni
clienti.query('Regione == "Lazio" and Nome == "Giovanni"') [["NumeroCliente", "Nome"]]

In [ ]:
#vecchio metodo
clienti[(clienti["Regione"] == "Lazio") & (clienti["Nome"] == "Giovanni")] \
       [["NumeroCliente", "Nome"]]

Possiamo anche dichiarare prima il valore da cercare

In [ ]:
x = "Lazio"
y = "Giovanni"

In [ ]:
clienti.query('Regione == @x and Nome == @y') [["NumeroCliente", "Nome"]]

## Filtri con OR

Estrarre NumeroCliente, Nome e regione dei clienti che soddisfano almeno
una delle seguenti proprietà:
1) sono della regione Lazio

2) si chiamano Giovanni

In [ ]:
#SELECT TOP 5 NumeroCliente,
#       Nome,
#       Regione
#FROM   Clienti 
#WHERE  Regione='Lazio'
#    OR Nome='Giovanni

clienti.query('Regione == "Lazio" or Nome == "Giovanni"') [["NumeroCliente", "Nome", "Regione"]].head(5)

## Filtri con isin

Estrarre  NumeroCliente, Nome e regione dei clienti residenti nel Lazio
o nel Piemonte

In [ ]:
#SELECT NumeroCliente, 
#       Nome, 
#       Regione
#FROM   Clienti 
#WHERE  Regione = 'Piemonte'
#    OR Regione='Lazio'
#oppure
#SELECT NumeroCliente, 
#       Nome, 
#       Regione
#FROM   Clienti 
#WHERE  Regione IN ('Piemonte', 'Lazio')

clienti.query('Regione in ["Piemonte","Lazio"]')[["NumeroCliente", "Nome","Regione"]]

## Attenzione ai tipi!

Se cerco una parola in una colonna di tipo intero, non otterrò risultati

In [ ]:
clienti.info()

In [ ]:
clienti.query("NumeroCliente == '3'")

Proviamo a convertire la colonna in stringa in un nuovo DataFrame

In [ ]:
clienti2 = clienti.copy()
clienti2["NumeroCliente"] = clienti2["NumeroCliente"].astype(str)

In [ ]:
clienti2.query("NumeroCliente == '3'")

Viceversa non otterrò risultati con la prossima query

In [ ]:
clienti2.query("NumeroCliente == 3 ")

## Attenzione ai null! 

Null risulta diverso da Lazio (non accade lo stesso con l'SQL)

In [ ]:
clienti.query("Regione != 'Lazio'").head(5)

estrarre tutte le righe dove la regione è null

In [ ]:
#SELECT *
#FROM   Clienti 
#WHERE  Regione IS NULL

clienti.query("Regione.isna()")

Estrarre 5 righe dove la regione non è null

In [ ]:
#SELECT TOP 5 *
#FROM   Clienti 
#WHERE  Regione IS NOT NULL

clienti.query("Regione.notna()").head(5)

## Filtri su colonne di tipo datetime

Estrarre tutte le fatture emesse dopo il 3 marzo 2018

In [ ]:
fatture.dtypes

In [ ]:
#SQL
#SELECT *
#FROM   Fatture
#WHERE  DataFattura > '2018-03-01'

fatture.query("DataFattura > '2018-03-01'")

Vediamo come utilizzare una variabile

In [ ]:
from datetime import datetime
data = datetime(2018,3,1)
fatture.query("DataFattura > @data")

## Filtri su funzioni applicate a colonne

Estrarre tutte le fatture del 2018

In [ ]:
#SELECT *
#FROM   Fatture
#WHERE  YEAR(DataFattura) = 2018

fatture.query("DataFattura.dt.year == 2018")

Estrarre tutti i clienti di nome Nicola, facendo una ricerca non case sensitive

In [ ]:
clienti.query("Nome.str.upper() == 'NICOLA'")

# Aggregare i dati

## Contare il numero di righe

Contare il numero di righe del dataframe Fatture

In [ ]:
#SELECT COUNT(*)
#FROM   Fatture

len(fatture)

In [ ]:
#oppure
fatture.shape[0]

## Calcolare indici statistici su tutte le colonne

Calcolare la somma di tutte le colonne

In [ ]:
fatture.sum(numeric_only=True)

Se volessi solo il dato di importo scriverei

In [ ]:
fatture.sum(numeric_only=True)["Importo"]

Calcolare i principali indici statistici di tutte le colonne

In [ ]:
fatture.describe()

Calcolare correlazioni tra colonne numeriche di un DataFrame

In [ ]:
fatture.corr(numeric_only=True)

## Aggregare su una colonna

Calcolare il numero di Fatture per ogni fornitore 

In [ ]:
#SELECT   NumeroFornitore, count(*)
#FROM     Fatture
#GROUP BY NumeroFornitore;

fatture.groupby(by="NumeroFornitore",
                as_index=False, 
                dropna=False
               )["NumeroFattura"].size()

Se eventualmente voglio cambiare il nome della colonna con il conteggio posso usare il metodo rename

In [ ]:
fatture.groupby(by="NumeroFornitore",
                as_index=False, 
                dropna=False)["NumeroFattura"].size().rename(columns={"size":"numero_totale"})

Altro metodo

In [ ]:
#SELECT   NumeroFornitore, count(*)
#FROM     Fatture
#GROUP BY NumeroFornitore;

fatture.groupby(by="NumeroFornitore",
                as_index=False, 
                dropna=False) \
       .agg(conteggio   = ("NumeroFattura",
                           np.size)
            )

I dati raggruppati si presentano ad essere rappresentati graficamente

In [ ]:
grafico = fatture.groupby(by="NumeroFornitore",
                          as_index=False, 
                          dropna=False).size() 

grafico.plot(x = 'NumeroFornitore',
             y = 'size',
             kind = 'bar')

## Altri esempi di aggregazione

Somma degli importi per ogni fornitore

In [ ]:
fatture.groupby(by=["NumeroFornitore"], 
                as_index=False,
                dropna=False)[["Importo"]].sum()

Media di importi e iva per ogni fornitore

In [ ]:
fatture.groupby(by=["NumeroFornitore"], 
                as_index=False, 
                dropna=False)[["Importo","Iva"]].mean().rename(columns={"Importo":"Importo_medio"})

Somma di importo e media di iva per ogni fornitore e cliente

In [ ]:
fatture.groupby(by=["NumeroFornitore","IdCliente"], as_index=False, dropna=False).\
        agg({"Importo":"sum","Iva":"mean"}).\
        rename(columns={"Importo":"Somma importo", "Iva":"Media iva"}).head(10)

Numero di clienti univoci

In [ ]:
fatture["IdCliente"].nunique()

Numero di clienti univoci al variare del fornitore

In [ ]:
fatture.groupby(by=["NumeroFornitore"], 
                as_index=False, dropna=False)["IdCliente"].nunique()

Numero di regioni presenti nel Dataframe dei clienti

In [ ]:
clienti["Regione"].nunique()

Elenco di regioni univoche

In [ ]:
clienti[["Regione"]].drop_duplicates()

# Combinare dataframe differenti

Riportare in un solo dataframe tutte le colonne dei dataframe Fatture e Clienti.

In [ ]:
fatture.head(1)

In [ ]:
#SELECT     TOP 5 *
#FROM       Fatture
#INNER JOIN Clienti
# ON Fatture.IdCliente = Clienti.NumeroCliente

f2 = pd.merge(fatture, 
         clienti, 
         how = 'inner',
         left_on = "IdCliente", 
         right_on = "NumeroCliente",
         suffixes = ('_fatture', '_clienti'))

In [ ]:
f2.head(1)

Esercizio: estrarre NumeroCliente e nome dei clienti senza fatture

In [ ]:
#SQL
#SELECT    TOP 5 Clienti.NumeroCliente, Clienti.Nome
#FROM      Clienti
#LEFT JOIN Fatture
#       ON Clienti.NumeroCliente = Fatture.IdCliente
#WHERE F.IdCliente IS NULL

pd.merge(clienti,
         fatture,
         how = 'left',
         left_on = "NumeroCliente",
         right_on = "IdCliente").query("IdCliente.isna()")[["NumeroCliente","Nome"]].head(5)


# Ordinare un dataframe

## Ordinare un dataframe per una colonna

Visualizzare le fatture dalla più recente alla meno recente

In [ ]:
#SQL
#SELECT TOP 5 *
#FROM   Fatture
#ORDER BY DataFattura DESC

fatture.sort_values(by=['Importo'], ascending=False).head(5)

Creare un nuovo dataframe con il nuovo ordine

In [ ]:
fatture_new = fatture.sort_values(by=['DataFattura'], ascending=False).copy()
fatture_new.head(5)

Resettiamo l'indice di FattureNew

In [ ]:
fatture_new = fatture_new.reset_index(drop=True)
fatture_new.head(5)

## Ordinare un dataframe in maniera randomica 

Visualizzare il dataframe Fatture con un ordinamento randomico

In [ ]:
fatture.sample(frac=1,
               random_state=0 #per rendere l'ordinamento ripetibile)
              ).head(5)

# Modificare un dataframe

## Copiare un dataframe

Copiare il dataframe Clienti in ClientiBis

In [ ]:
#SQL
#SELECT *
#INTO   ClientiBis
#FROM   Clienti

clienti_bis = clienti.copy()

## Convertire un dataframe in un array di numpy

Creare un array di numpy a partire dal dataframe clienti

In [ ]:
array = clienti.values

In [ ]:
type(array)

In [ ]:
array[0:4]

## Convertire un dataframe in una lista di liste

Convertire il dataframe Clienti in una lista

In [ ]:
lista_clienti = clienti.values.tolist()

In [ ]:
type(lista_clienti)

In [ ]:
lista_clienti[0:4]

## Creare un dataframe a partire da una lista

Creare un dataframe a partire da una lista, assegnando i nomi alle colonne

In [ ]:
new_df = pd.DataFrame(data = lista_clienti, 
                     columns = ['NumeroCliente', 'Nome', 'Cognome', 'DataNascita', 'Regione'] )

In [ ]:
new_df.dtypes

## Rinominare una colonna 

Rinominare la colonna Iva del dataframe Fatture in Tax

In [ ]:
fatture = fatture.rename(columns={"Iva": "Tax"})
fatture.head(3)

## Eliminare una colonna 

Eliminare la colonna NumeroFornitore dal dataframe Fatture

In [ ]:
fatture.columns

In [ ]:
#SQL
#ALTER TABLE Fatture
#DROP COLUMN NumeroFornitore

fatture = fatture.drop(["NumeroFornitore"], axis=1)
#fatture.head(3)

## Aggiornare una colonna

Guardiamo i clienti della regione Lazio

In [ ]:
clienti.query("Regione == 'Lazio'")

Modificare il nome di tutti i clienti della regione Lazio valorizzandolo con "Nicola"

In [ ]:
#SQL
#UPDATE Clienti
#SET    Nome = 'Nicola'
#WHERE  Regione = 'Lazio'

clienti.loc[clienti["Regione"] == "Lazio", "Nome"] = 'Nicola'

In [ ]:
clienti.query("Regione == 'Lazio'")

In [ ]:
#oppure

clienti["Nome"] = np.where(clienti["Regione"] == "Lazio", 
                           "Nicola", 
                           clienti["Nome"]  )

## Aggiornare una colonna sostituendo i null con un valore

Sostituire i null presenti nella colonna Regione del dataframe Cliente con la stringa "Non conosciuta"

In [ ]:
#SQL
#UPDATE Clienti
#SET    Regione = COALESCE(Regione,'Non conosciuta')
#WHERE  Regione IS NULL

clienti["Regione"] = clienti["Regione"].fillna("Non conosciuta")

In [ ]:
clienti.head(3)

## Concatenare una colonna

Concatenare le colonne Nazione e Regione del dataframe clienti.

Attenzione, in presenza di un null il risultato della concatenazione sarà null. Per evitare che ciò accada, il metodo fillna può essere usato per sostituire i null con ''.

In [ ]:
clienti["ColonnaConcatenata"] = clienti["Nazione"].fillna("") + "," + clienti["Regione"].fillna("")
clienti.head(3)

## Splittare una colonna

Splittare tramite la virgola la colonna appena creata in due colonne: ColonnaSplit1 e ColonnaSplit2

In [ ]:
clienti[["ColonnaSplit1","ColonnaSplit2"]] = clienti["ColonnaConcatenata"].str.split(pat = ',', expand=True)
clienti.head(3)

## Creare una colonna in base ad una condizione

Creare nel dataframe Fatture la colonna TipologiaPrezzo contenente "Prezzo alto" se il valore della colonna Importo è maggiore di 50, "Prezzo basso" altrimenti.

In [ ]:
#SQL
#CASE WHEN Importo > 50 
#     THEN 'Prezzo alto'
#     ELSE 'Prezzo basso'
#END

fatture["TipologiaPrezzo"] = np.where(fatture["Importo"] > 50, "Prezzo alto", "Prezzo basso")
fatture.head(5)

## Creare una colonna in base a più condizioni

Creare nel dataframe Fatture la colonna TipologiaPrezzo contenente:

- "Prezzo alto" se il valore della colonna Importo è maggiore di 50 

- "Prezzo medio" se è compreso tra 30(escluso) e 50(incluso)

- "Prezzo basso" se minore o uguale a 30

In [ ]:
fatture["TipologiaPrezzo2"] = np.select([fatture["Importo"] > 50, 
                                         (fatture["Importo"] > 30) & (fatture["Importo"] <=50),
                                         fatture["Importo"] <= 30],
                                         ["Prezzo alto", 
                                          "Prezzo medio",
                                          "Prezzo basso"],
                                          default='Non classificato')
fatture.head(5)

## Creare colonne con porzioni di stringhe di un'altra colonna

Creare una colonna con le iniziali di nome e cognome per ogni cliente

In [ ]:
#SQL
#SUBSTRING(Nome,1,1)

clienti["Iniziale"] =  clienti["Nome"].str.slice(start=0, stop =1)
clienti.head(3)

## Visualizzare i dati senza duplicati

Il prossimo codice visualizza le righe del DataFrame clienti senza duplicati (considerando i valori in tutte le colonne). 

ATTENZIONE! Per DataFrame con molte righe può essere un'operazione molto lunga

In [ ]:
clienti.drop_duplicates().head(3)

Più utile è utilizzare drop_duplicates per ottenere l'elenco di valori distinti in una colonna (o combinazione di colonne)

In [ ]:
#Elenco delle regioni presenti: primo metodo

clienti[["Regione"]].drop_duplicates()

In [ ]:
#Elenco delle regioni presenti: secondo metodo

clienti.drop_duplicates(subset=["Regione"], keep = "first")

In [ ]:
#Proviamo a ordinare prima il DataFrame

In [ ]:
clienti.sort_values(by="DataNascita",ascending=False).drop_duplicates(subset=["Regione"], keep = "first")

## Pivot e unpivot dei dati

Raggruppiamo i dati per cliente e tipologia

In [ ]:
df = fatture.groupby(by=["IdCliente","Tipologia"], as_index=False, dropna=False)["Importo"].sum()

df

Modifichiamo la forma dell'output, visualizziamo una per ogni cliente due colonne per gli importi in A e V

In [ ]:
df_pivot = df.pivot(columns = "Tipologia",  #da quale colonna dell'input costruire le colonne dell'output
                    index = "IdCliente",    #quale sarà l'indice del nuovo dataframe
                    values = "Importo" #cosa riportare nelle righe
                    )
df_pivot

L'IdCliente è l'indice del dataframe

In [ ]:
df_pivot.index

Rendiamolo una colonna

In [ ]:
df_pivot.reset_index()

eliminiamo il nome dell'indice

In [ ]:
df_pivot = df_pivot.reset_index().rename_axis(None, axis=1)
df_pivot

Torniamo alla visualizzazione con le colonne IdCliente e Tipologia con unpivot

In [ ]:
df_unpivot = pd.melt(df_pivot, 
        id_vars='IdCliente', #colonna da lasciare nell'outoput
        value_vars=list(df_pivot.columns).remove("IdCliente"), #colonne da trasformare in righe
        var_name='Tipologia', #nome della nuova colonna contenente le vecche colonne 
        value_name='Importo' #nome della nuova colonna contenente i valori
                    )

df_unpivot